## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-13-17-08-39 +0000,nypost,DOJ sues Harvard for withholding records on al...,https://nypost.com/2026/02/13/us-news/doj-sues...
1,2026-02-13-17-06-00 +0000,wsj,Justice Department Sues Harvard Over Race Admi...,https://www.wsj.com/us-news/education/trump-do...
2,2026-02-13-17-05-09 +0000,nypost,"Ukrainian brothers, including 8-year-old, kill...",https://nypost.com/2026/02/13/world-news/russi...
3,2026-02-13-17-00-00 +0000,wsj,How Popcorn Took Over the Movies,https://www.wsj.com/arts-culture/food-cooking/...
4,2026-02-13-17-00-00 +0000,missionlocal,Love on aisle three: Inside a Valentine’s mixe...,https://missionlocal.org/2026/02/sf-trader-joe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2527/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
136,trump,39
129,epstein,18
166,new,14
728,immigration,12
952,shooting,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
304,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...,111
127,2026-02-13-12-00-00 +0000,wsj,Republican Sen. Rand Paul joined Democrats in ...,https://www.wsj.com/politics/gop-sen-rand-paul...,98
175,2026-02-13-10-00-16 +0000,wapo,Services cut. Staff laid off. New Trump cuts h...,https://www.washingtonpost.com/education/2026/...,85
241,2026-02-13-01-44-10 +0000,latimes,Judge blocks Trump administration move to cut ...,https://www.latimes.com/politics/story/2026-02...,81
356,2026-02-12-19-47-08 +0000,bbc,Minnesota immigration enforcement surge is end...,https://www.bbc.com/news/articles/c2lr9w29zwyo...,77


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
304,111,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...
265,72,2026-02-12-23-36-00 +0000,wsj,"Chris Klomp, the head of Medicare and a former...",https://www.wsj.com/politics/policy/white-hous...
273,66,2026-02-12-23-16-31 +0000,nypost,Here’s the real reason Goldman CEO David Solom...,https://nypost.com/2026/02/12/business/heres-t...
26,56,2026-02-13-16-18-00 +0000,wsj,What a shutdown would mean for the Department ...,https://www.wsj.com/politics/policy/homeland-s...
108,50,2026-02-13-13-04-33 +0000,nypost,Trans shooter Jesse Van Rootselaar — who kille...,https://nypost.com/2026/02/13/world-news/jesse...
183,49,2026-02-13-09-24-45 +0000,nypost,Arizona Sheriff Chris Nanos fires back at clai...,https://nypost.com/2026/02/13/us-news/arizona-...
267,43,2026-02-12-23-33-00 +0000,wsj,When Customs and Border Protection fired a las...,https://www.wsj.com/politics/national-security...
327,42,2026-02-12-21-05-00 +0000,wsj,Judge Blocks Pentagon’s Move to Censure Sen. M...,https://www.wsj.com/us-news/law/judge-blocks-p...
395,41,2026-02-12-17-59-02 +0000,nypost,Gail Slater leaving role as DOJ antitrust chie...,https://nypost.com/2026/02/12/business/gail-sl...
38,39,2026-02-13-15-57-09 +0000,nypost,Watch out for these 5 hidden hotel charges — a...,https://nypost.com/2026/02/13/lifestyle/watch-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
